<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/04_SRTM90_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process DEM to derive topographical indices


## Import gee and authenticate

Expiry period: the granted permissions will expire in a week, after which you'll need to call ee.Authenticate() again.

In [ ]:
# This doesn't really work, it's only valid for the current runtime...
import os

# If credentials file doesn't exist, authenticate and store credentials
# Else if credentials file does exist, use stored credentials and initialise
if not os.path.exists(os.path.expanduser("~/.config/earthengine/credentials")):
  import ee
  ee.Authenticate()
  ee.Initialize()
else:
  import ee
  ee.Initialize()

## Import AOI and set variable names


In [ ]:
# Area of interest
# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
# aoi_name = "MKR_NS_buff_5km"

aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"

# aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/carbon_rs_aoi")
# aoi_name = "carbon_rs_aoi"

elev90m_layer_name = "RS_005"
slope_layer_name = "RS_006"
aspect_layer_name = "RS_007"
elev30m_layer_name = "RS_072"

## Import image and calculate topography metrics


In [ ]:
elevation_90 = ee.Image("CGIAR/SRTM90_V4").select("elevation").clip(aoi)
slope = ee.Terrain.slope(elevation_90)
aspect = ee.Terrain.aspect(elevation_90)
elevation_30 = ee.Image('USGS/SRTMGL1_003').select("elevation").clip(aoi)

## Export data - create tasks

In [ ]:
task1 = ee.batch.Export.image.toDrive(image = elevation_90,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = elev90m_layer_name + "_" + aoi_name,
                                     scale = 90,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task1.start()

In [ ]:
task2 = ee.batch.Export.image.toDrive(image = slope,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = slope_layer_name + "_" + aoi_name,
                                     scale = 90,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task2.start()

In [ ]:
task3 = ee.batch.Export.image.toDrive(image = aspect,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = aspect_layer_name + "_" + aoi_name,
                                     scale = 90,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )

task3.start()

In [ ]:
task4 = ee.batch.Export.image.toDrive(image = elevation_30,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = elev30m_layer_name + "_" + aoi_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
task4.start()

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:3]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)